In [72]:
import os
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DirectionModel(nn.Module):
    def __init__(self, n_features, n_classes):
        super(DirectionModel, self).__init__()
        self.fc1 = nn.Linear(n_features, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(512, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.5)
        
        self.fc4 = nn.Linear(64, n_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout2(x)
        
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = self.dropout3(x)
        
        x = self.fc4(x)
        return x
model = DirectionModel(3, 5)

In [74]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
device

'cpu'

In [75]:
model_path = "direction_model.pt"
model = torch.load(model_path).to(device)
model.eval()

DirectionModel(
  (fc1): Linear(in_features=3, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (fc4): Linear(in_features=64, out_features=5, bias=True)
)

In [88]:
train_df = pd.read_csv("affectnet_train.csv", names=['class', 'filename', 'pitch', 'yaw', 'roll'])
train_df.head()

,class,filename,pitch,yaw,roll
0,1,image0012261.jpg,-0.214550,0.152226,0.103692
1,1,image0003195.jpg,0.207676,-0.255355,0.036119
2,1,image0010070.jpg,-0.204092,0.133176,-0.009492
3,1,image0017195.jpg,0.009019,0.060406,-0.076559
4,1,image0014177.jpg,-0.001475,0.212992,0.076477


In [89]:
train_X = train_df.iloc[:, 2:]
train_X.head()

,pitch,yaw,roll
0,-0.214550,0.152226,0.103692
1,0.207676,-0.255355,0.036119
2,-0.204092,0.133176,-0.009492
3,0.009019,0.060406,-0.076559
4,-0.001475,0.212992,0.076477


In [90]:
train_X = torch.tensor(train_X.values, dtype=torch.float32).to(device)

In [91]:
train_dataloader = DataLoader(train_X, shuffle=False)

Can it be done without without all the steps? Maybe...

In [92]:
model.eval()
preds = []
with torch.no_grad():
        for i, inputs in enumerate(train_dataloader):
            predicted = torch.argmax(model(inputs))
            preds.append(predicted.item())

In [93]:
train_df['direction'] = preds

In [94]:
train_df.head()

,class,filename,pitch,yaw,roll,direction
0,1,image0012261.jpg,-0.214550,0.152226,0.103692,2
1,1,image0003195.jpg,0.207676,-0.255355,0.036119,2
2,1,image0010070.jpg,-0.204092,0.133176,-0.009492,0
3,1,image0017195.jpg,0.009019,0.060406,-0.076559,1
4,1,image0014177.jpg,-0.001475,0.212992,0.076477,2


In [95]:
train_df.to_csv("affectnet_train_model_direction.csv", index=False)

In [96]:
val_df = pd.read_csv("affectnet_val.csv", names=['class', 'filename', 'pitch', 'yaw', 'roll'])
val_df.head()

,class,filename,pitch,yaw,roll
0,1,image0004172.jpg,-0.101674,0.260372,0.026084
1,1,image0002985.jpg,0.002530,0.100325,0.034182
2,1,image0002507.jpg,0.126349,0.085475,-0.175663
3,1,image0000594.jpg,0.196749,0.049497,0.006675
4,1,image0003238.jpg,-0.083720,0.230250,0.083975


In [97]:
val_X = val_df.iloc[:, 2:]
val_X.head()

,pitch,yaw,roll
0,-0.101674,0.260372,0.026084
1,0.002530,0.100325,0.034182
2,0.126349,0.085475,-0.175663
3,0.196749,0.049497,0.006675
4,-0.083720,0.230250,0.083975


In [98]:
val_X = torch.tensor(val_X.values, dtype=torch.float32).to(device)

In [101]:
val_dataloader = DataLoader(val_X, shuffle=False)

In [102]:
model.eval()
preds = []
with torch.no_grad():
        for i, inputs in enumerate(val_dataloader):
            predicted = torch.argmax(model(inputs))
            preds.append(predicted.item())

In [103]:
val_df['direction'] = preds

In [104]:
val_df.head()

,class,filename,pitch,yaw,roll,direction
0,1,image0004172.jpg,-0.101674,0.260372,0.026084,2
1,1,image0002985.jpg,0.002530,0.100325,0.034182,0
2,1,image0002507.jpg,0.126349,0.085475,-0.175663,1
3,1,image0000594.jpg,0.196749,0.049497,0.006675,1
4,1,image0003238.jpg,-0.083720,0.230250,0.083975,2


In [105]:
val_df.to_csv("affectnet_val_model_direction.csv", index=False)